### Import Required Libraries and Set Up Environment Variables

In [72]:
# Dependencies
import requests
import time
from dotenv import load_dotenv
import os
import pandas as pd
import json

In [73]:
#debug constant that will have various values print if TRUE
debug = False

In [74]:
# Set environment variables from the .env in the local environment
load_dotenv()

nyt_api_key = os.getenv("NYT_API_KEY")
tmdb_api_key = os.getenv("TMDB_API_KEY")

if debug:
    print(nyt_api_key)
    print(tmdb_api_key)

### Access the New York Times API

In [75]:
# Set the base URL
url = "https://api.nytimes.com/svc/search/v2/articlesearch.json?"

# Filter for movie reviews with "love" in the headline
# section_name should be "Movies"
# type_of_material should be "Review"
filter_query_character = "&fq="
filter_query = 'section_name:"Movies" AND type_of_material:"Review" AND headline:"love"'

# Use a sort filter, sort by newest
sort_character = "&sort="
sort = "newest"

# Select the following fields to return:
# headline, web_url, snippet, source, keywords, pub_date, byline, word_count
field_list_character = "&fl="
field_list = "headline,web_url,snippet,source,keywords,pub_date,byline,word_count"

# Search for reviews published between a begin and end date
begin_date_character = "&begin_date="
end_date_character = "&end_date="
begin_date = "20130101"
end_date = "20230531"


page_character = "&page="
api_key_character = "&api-key="

# Build URL
base_query_url = url +\
                filter_query_character + filter_query +\
                sort_character + sort +\
                field_list_character + field_list +\
                begin_date_character + begin_date +\
                end_date_character + end_date

print("base query url=",base_query_url)


base query url= https://api.nytimes.com/svc/search/v2/articlesearch.json?&fq=section_name:"Movies" AND type_of_material:"Review" AND headline:"love"&sort=newest&fl=headline,web_url,snippet,source,keywords,pub_date,byline,word_count&begin_date=20130101&end_date=20230531


In [77]:
# Create an empty list to store the reviews
reviews = []

# loop through pages 0-19
for page_number in range(0,20):
    # create query with a page number
    # API results show 10 articles at a time
    current_query_url = base_query_url + page_character + f"{page_number}" + api_key_character + nyt_api_key

    # Make a "GET" request and retrieve the JSON
    nyt_request = requests.get(current_query_url)

    # get the start time for the twelve second timer
    start_time = time.time()
    
    if nyt_request.status_code == 200:
        nyt_json = nyt_request.json()
        
        # Try and save the reviews to the reviews_list
        for review in nyt_json["response"]["docs"]:
            # loop through the reviews["response"]["docs"] and append each review to the list
            reviews.append(review)

        # Print the page that was just retrieved
        # Print the page number that had no results then break from the loop
        string_to_print = f"Checked page {page_number}"
        if debug:
            string_to_print = string_to_print + ", Good Response"

        print(string_to_print)

        
    else:
        print(f"Checked page {page_number}, Bad Response")
        print(f"     NYT Get Request Failed, error code = {nyt_request.status_code}")

    # Add a twelve second interval between queries to stay within API query limits
    end_time = time.time()
    elapsed_time = end_time - start_time
    delay_time = max(0.0, 12-elapsed_time)
    time.sleep(delay_time)

if debug:
    print(len(reviews))

Checked page 0
Checked page 1
Checked page 2
Checked page 3
Checked page 4
Checked page 5
Checked page 6
Checked page 7
Checked page 8
Checked page 9
Checked page 10
Checked page 11
Checked page 12
Checked page 13
Checked page 14
Checked page 15
Checked page 16
Checked page 17
Checked page 18
Checked page 19


In [78]:
# Preview the first 5 results in JSON format
# Use json.dumps with argument indent=4 to format data
for i in range(0,5):
    print(json.dumps(reviews[i], indent=4))


{
    "web_url": "https://www.nytimes.com/2023/05/25/movies/the-attachment-diaries-review.html",
    "snippet": "A gynecologist and her patient form a horrifyingly twisted connection in this batty, bloody Argentine melodrama.",
    "source": "The New York Times",
    "headline": {
        "main": "\u2018The Attachment Diaries\u2019 Review: Love, Sick",
        "kicker": null,
        "content_kicker": null,
        "print_headline": "The Attachment Diaries",
        "name": null,
        "seo": null,
        "sub": null
    },
    "keywords": [
        {
            "name": "subject",
            "value": "Movies",
            "rank": 1,
            "major": "N"
        },
        {
            "name": "creative_works",
            "value": "The Attachment Diaries (Movie)",
            "rank": 2,
            "major": "N"
        },
        {
            "name": "persons",
            "value": "Diment, Valentin Javier",
            "rank": 3,
            "major": "N"
        }
    ],
  

In [79]:
# Convert reviews_list to a Pandas DataFrame using json_normalize()
reviews_df = pd.json_normalize(reviews)
reviews_df.head()

,web_url,snippet,source,keywords,pub_date,word_count,headline.main,headline.kicker,headline.content_kicker,headline.print_headline,headline.name,headline.seo,headline.sub,byline.original,byline.person,byline.organization
0,https://www.nytimes.com/2023/05/25/movies/the-...,A gynecologist and her patient form a horrifyi...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-05-25T11:00:03+0000,295,"‘The Attachment Diaries’ Review: Love, Sick",None,None,The Attachment Diaries,None,None,None,By Jeannette Catsoulis,"[{'firstname': 'Jeannette', 'middlename': None...",None
1,https://www.nytimes.com/2023/05/04/movies/what...,Two childhood friends navigate cultural differ...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-05-04T17:16:45+0000,287,Review: ‘What’s Love Got to Do With It?’ Proba...,None,None,What’s Love Got to Do With It?,None,None,None,By Jeannette Catsoulis,"[{'firstname': 'Jeannette', 'middlename': None...",None
2,https://www.nytimes.com/2023/05/04/movies/you-...,Religion comes between two girls falling in lo...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-05-04T11:00:08+0000,294,‘You Can Live Forever’ Review: Do You Love Me ...,None,None,You Can Live Forever,None,None,None,By Elisabeth Vincentelli,"[{'firstname': 'Elisabeth', 'middlename': None...",None
3,https://www.nytimes.com/2023/04/21/movies/a-to...,Rachael Leigh Cook stars in this bland rom-com...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-04-21T07:03:25+0000,276,‘A Tourist’s Guide to Love’ Review: A Wearying...,None,None,A Tourist’s Guide to Love,None,None,None,By Elisabeth Vincentelli,"[{'firstname': 'Elisabeth', 'middlename': None...",None
4,https://www.nytimes.com/2023/04/20/movies/othe...,A radiant Virginie Efira stars as a Parisian t...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-04-20T15:35:13+0000,801,‘Other People’s Children’ Review: True Romance,Critic’s pick,None,Intoxicating Love With a Sobering Turn,None,None,None,By Manohla Dargis,"[{'firstname': 'Manohla', 'middlename': None, ...",None


In [80]:
# Extract the title from the "headline.main" column and
# save it to a new column "title"
# Title is between unicode characters \u2018 and \u2019. 
# End string should include " Review" to avoid cutting title early

left_quote = u"\u2018" #‘
right_quote = u"\u2019" #’

title_column = []

for headline in reviews_df["headline.main"]:
    start_char = ""
    end_char = ""

    if left_quote in str(headline) and right_quote in str(headline):
        start_char = u"\u2018" #‘
        end_char = u"\u2019" #’
    elif "Review: " in str(headline) and " —" in str(headline):
        start_char = "Review: "
        end_char = " —"

    start_index = str(headline).index(start_char)
    end_index = str(headline).rindex(end_char)


    extracted_title = str(headline)[start_index + len(start_char): end_index]

    #code to fix problems in specific titles I am only able to do this because 
    #the list of titles is small enough that I can look through them for weird outliers manually
    if f"{right_quote} Review" in extracted_title:
        extracted_title = extracted_title.split(f"{right_quote} Review")[0]
    
    if f"{right_quote} Documents" in extracted_title:
        extracted_title = extracted_title.split(f"{right_quote} Documents")[0]

    if f",{right_quote}" in extracted_title:
        extracted_title = extracted_title.split(f",{right_quote}")[0]
        
    title_column.append(extracted_title)

reviews_df["title"] = title_column

if debug:
    for title in title_column:
        print(title)
    display(reviews_df.loc[199:199])
    print(reviews_df["headline.main"][199])
    print(title_column[199])

reviews_df.head()


,web_url,snippet,source,keywords,pub_date,word_count,headline.main,headline.kicker,headline.content_kicker,headline.print_headline,headline.name,headline.seo,headline.sub,byline.original,byline.person,byline.organization,title
0,https://www.nytimes.com/2023/05/25/movies/the-...,A gynecologist and her patient form a horrifyi...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-05-25T11:00:03+0000,295,"‘The Attachment Diaries’ Review: Love, Sick",None,None,The Attachment Diaries,None,None,None,By Jeannette Catsoulis,"[{'firstname': 'Jeannette', 'middlename': None...",None,The Attachment Diaries
1,https://www.nytimes.com/2023/05/04/movies/what...,Two childhood friends navigate cultural differ...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-05-04T17:16:45+0000,287,Review: ‘What’s Love Got to Do With It?’ Proba...,None,None,What’s Love Got to Do With It?,None,None,None,By Jeannette Catsoulis,"[{'firstname': 'Jeannette', 'middlename': None...",None,What’s Love Got to Do With It?
2,https://www.nytimes.com/2023/05/04/movies/you-...,Religion comes between two girls falling in lo...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-05-04T11:00:08+0000,294,‘You Can Live Forever’ Review: Do You Love Me ...,None,None,You Can Live Forever,None,None,None,By Elisabeth Vincentelli,"[{'firstname': 'Elisabeth', 'middlename': None...",None,You Can Live Forever
3,https://www.nytimes.com/2023/04/21/movies/a-to...,Rachael Leigh Cook stars in this bland rom-com...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-04-21T07:03:25+0000,276,‘A Tourist’s Guide to Love’ Review: A Wearying...,None,None,A Tourist’s Guide to Love,None,None,None,By Elisabeth Vincentelli,"[{'firstname': 'Elisabeth', 'middlename': None...",None,A Tourist’s Guide to Love
4,https://www.nytimes.com/2023/04/20/movies/othe...,A radiant Virginie Efira stars as a Parisian t...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-04-20T15:35:13+0000,801,‘Other People’s Children’ Review: True Romance,Critic’s pick,None,Intoxicating Love With a Sobering Turn,None,None,None,By Manohla Dargis,"[{'firstname': 'Manohla', 'middlename': None, ...",None,Other People’s Children


In [81]:
# Extract 'name' and 'value' from items in "keywords" column
def extract_keywords(keyword_list):
    extracted_keywords = ""
    for item in keyword_list:
        # Extract 'name' and 'value'
        keyword = f"{item['name']}: {item['value']};" 
        # Append the keyword item to the extracted_keywords list
        extracted_keywords += keyword
    return extracted_keywords

# Fix the "keywords" column by converting cells from a list to a string
reviews_df["keywords"] = reviews_df["keywords"].apply(lambda x: extract_keywords(x))
reviews_df.head()

,web_url,snippet,source,keywords,pub_date,word_count,headline.main,headline.kicker,headline.content_kicker,headline.print_headline,headline.name,headline.seo,headline.sub,byline.original,byline.person,byline.organization,title
0,https://www.nytimes.com/2023/05/25/movies/the-...,A gynecologist and her patient form a horrifyi...,The New York Times,subject: Movies;creative_works: The Attachment...,2023-05-25T11:00:03+0000,295,"‘The Attachment Diaries’ Review: Love, Sick",None,None,The Attachment Diaries,None,None,None,By Jeannette Catsoulis,"[{'firstname': 'Jeannette', 'middlename': None...",None,The Attachment Diaries
1,https://www.nytimes.com/2023/05/04/movies/what...,Two childhood friends navigate cultural differ...,The New York Times,"subject: Movies;persons: Kapur, Shekhar;person...",2023-05-04T17:16:45+0000,287,Review: ‘What’s Love Got to Do With It?’ Proba...,None,None,What’s Love Got to Do With It?,None,None,None,By Jeannette Catsoulis,"[{'firstname': 'Jeannette', 'middlename': None...",None,What’s Love Got to Do With It?
2,https://www.nytimes.com/2023/05/04/movies/you-...,Religion comes between two girls falling in lo...,The New York Times,subject: Movies;creative_works: You Can Live F...,2023-05-04T11:00:08+0000,294,‘You Can Live Forever’ Review: Do You Love Me ...,None,None,You Can Live Forever,None,None,None,By Elisabeth Vincentelli,"[{'firstname': 'Elisabeth', 'middlename': None...",None,You Can Live Forever
3,https://www.nytimes.com/2023/04/21/movies/a-to...,Rachael Leigh Cook stars in this bland rom-com...,The New York Times,subject: Movies;creative_works: A Tourist's Gu...,2023-04-21T07:03:25+0000,276,‘A Tourist’s Guide to Love’ Review: A Wearying...,None,None,A Tourist’s Guide to Love,None,None,None,By Elisabeth Vincentelli,"[{'firstname': 'Elisabeth', 'middlename': None...",None,A Tourist’s Guide to Love
4,https://www.nytimes.com/2023/04/20/movies/othe...,A radiant Virginie Efira stars as a Parisian t...,The New York Times,"subject: Movies;persons: Zlotowski, Rebecca;cr...",2023-04-20T15:35:13+0000,801,‘Other People’s Children’ Review: True Romance,Critic’s pick,None,Intoxicating Love With a Sobering Turn,None,None,None,By Manohla Dargis,"[{'firstname': 'Manohla', 'middlename': None, ...",None,Other People’s Children


In [83]:
# Create a list from the "title" column using to_list()
# These titles will be used in the query for The Movie Database
nyt_titles = reviews_df["title"].to_list()
for title in nyt_titles:
    print(title)

The Attachment Diaries
What’s Love Got to Do With It?
You Can Live Forever
A Tourist’s Guide to Love
Other People’s Children
One True Loves
The Lost Weekend: A Love Story
A Thousand and One
Your Place or Mine
Love in the Time of Fentanyl
Pamela, a Love Story
In From the Side
After Love
Alcarràs
Nelly & Nadine
Lady Chatterley’s Lover
The Sound of Christmas
The Inspection
Bones and All
My Policeman
About Fate
Waiting for Bojangles
I Love My Dad
A Love Song
Alone Together
Art of Love
The Wheel
Thor: Love and Thunder
Both Sides of the Blade
Fire of Love
Love & Gelato
Stay Prayed Up
Benediction
Dinner in America
In a New York Minute
Anaïs in Love
I Love America
See You Then
La Mami
Love After Love
Deep Water
Lucy and Desi
Cyrano
The In Between
Book of Love
Lingui, the Sacred Bonds
The Pink Cloud
A Journal for Jordan
West Side Story
Aulcie
Love Is Love Is Love
Love Hard
Bergman Island
Hard Luck Love Song
South of Heaven
Wife of a Spy
Happier Than Ever
Together
Annette
Resort to Love
Woodstoc

### Access The Movie Database API

In [99]:
# Prepare The Movie Database query
url = "https://api.themoviedb.org/3/search/movie?query="
tmdb_key_string = "&api_key=" + tmdb_api_key

# These are for generating the query on the movie details
tmdb_base_details_url = "https://api.themoviedb.org/3/movie/"
tmdb_details_key_string = "?api_key=" + tmdb_api_key

In [111]:
# Create an empty list to store the results
tmdb_movies_list = []

# Create a request counter to sleep the requests after a multiple
# of 50 requests
request_counter = 1

# Loop through the titles
for title in nyt_titles:
    # Check if we need to sleep before making a request
    if (request_counter % 50) == 0:
        time.sleep(1)
        print(f"request {request_counter}, sleeping for 1 second")


    # Add 1 to the request counter
    request_counter += 1
    
    # Perform a "GET" request for The Movie Database
    tmdb_search_request = requests.get(url + title + tmdb_key_string)

    # Include a try clause to search for the full movie details.
    # Use the except clause to print out a statement if a movie
    # is not found.
    #if True:
    try:
        tmdb_search_json = tmdb_search_request.json()
        tmdb_search_json = tmdb_search_request.json()
        closest_movie = tmdb_search_json["results"][0]

        # Get movie id
        movie_id = closest_movie["id"]

        # Make a request for a the full movie details
        tmdb_details_url = tmdb_base_details_url + f"{movie_id}" + tmdb_details_key_string

        # Execute "GET" request with url
        tmdb_details_request = requests.get(tmdb_details_url)
        
        tmdb_details_json = tmdb_details_request.json()

        # Extract the genre names into a list
        genre_list = []
        
        for genre in tmdb_details_json["genres"]:
            genre_list.append(genre["name"])

        # Extract the spoken_languages' English name into a list
        spoken_language_list = []
        for spoken_language in tmdb_details_json["spoken_languages"]:
            spoken_language_list.append(spoken_language["english_name"])

        # Extract the production_countries' name into a list
        production_countries_list = []
        for production_country in tmdb_details_json["production_countries"]:
            production_countries_list.append(production_country["name"])

        # Add the relevant data to a dictionary and
        # append it to the tmdb_movies_list list
        movie_entry = {}
        movie_entry["title"] = tmdb_details_json["title"]
        movie_entry["original_title"] = tmdb_details_json["original_title"]
        movie_entry["budget"] = tmdb_details_json["budget"]
        movie_entry["genre"] = genre_list
        movie_entry["language"] = tmdb_details_json["original_language"]
        movie_entry["spoken_language"] = spoken_language_list
        movie_entry["homepage"] = tmdb_details_json["homepage"]
        movie_entry["overview"] = tmdb_details_json["overview"]
        movie_entry["popularity"] = tmdb_details_json["popularity"]
        movie_entry["runtime"] = tmdb_details_json["runtime"]
        movie_entry["revenue"] = tmdb_details_json["revenue"]
        movie_entry["release_date"] = tmdb_details_json["release_date"]
        movie_entry["vote_average"] = tmdb_details_json["vote_average"]
        movie_entry["vote_count"] = tmdb_details_json["vote_count"]
        movie_entry["production_countries"] = production_countries_list

        tmdb_movies_list.append(movie_entry)

        # Print out the title that was found
        print("Found", movie_entry["title"])
    #else:
    except:
        print(title,"not found.")



request 0, sleeping for 1 second
Found The Attachment Diaries
Found What's Love Got to Do with It?
Found You Can Live Forever
Found A Tourist's Guide to Love
Found Other People's Children
Found One True Loves
Found The Lost Weekend: A Love Story
Found A Thousand and One
Found Your Place or Mine
Found Love in the Time of Fentanyl
Found Pamela, A Love Story
Found In from the Side
Found After Love
Found Alcarràs
Found Nelly and Monsieur Arnaud
Found Lady Chatterley's Lover
Found The Sound of Christmas
Found The Inspection
Found Bones and All
Found My Policeman
Found About Fate
Found Waiting for Bojangles
Found I Love My Dad
Found A Love Song
Found Alone Together
Found The Art of Love
Found 2099: The Soldier Protocol
Found Thor: Love and Thunder
Found Both Sides of the Blade
Found Fire of Love
Found Sweet Sex and Love
Found Stay Prayed Up
Found Benediction
Found Dinner in America
Found In a New York Minute
Found Anaïs in Love
Found I Love America
Found See You Then
Found La Mami
Found Love

In [112]:
# Preview the first 5 results in JSON format
# Use json.dumps with argument indent=4 to format data
for i in range(0,5):
    print(json.dumps(tmdb_movies_list[i], indent=4))

{
    "title": "The Attachment Diaries",
    "original_title": "El apego",
    "budget": 0,
    "genre": [
        "Drama",
        "Mystery",
        "Thriller",
        "Horror"
    ],
    "language": "es",
    "spoken_language": [
        "Spanish"
    ],
    "homepage": "",
    "overview": "Argentina, 1970s. A desperate young woman goes to a clinic to have a clandestine abortion. As her pregnancy is already through the fourth month, the doctor refuses. Instead, she proposes to sell the baby to one of her clients and offers to provide shelter in her house until the child is born. Their disturbed personalities will become intertwined in a strange and dangerous relationship.",
    "popularity": 0.997,
    "runtime": 102,
    "revenue": 0,
    "release_date": "2021-10-07",
    "vote_average": 4.0,
    "vote_count": 2,
    "production_countries": [
        "Argentina"
    ]
}
{
    "title": "What's Love Got to Do with It?",
    "original_title": "What's Love Got to Do with It?",
    "bu

In [114]:
# Convert the results to a DataFrame
tmdb_movies_df = pd.DataFrame(tmdb_movies_list)
tmdb_movies_df.head()

,title,original_title,budget,genre,language,spoken_language,homepage,overview,popularity,runtime,revenue,release_date,vote_average,vote_count,production_countries
0,The Attachment Diaries,El apego,0,"[Drama, Mystery, Thriller, Horror]",es,[Spanish],,"Argentina, 1970s. A desperate young woman goes...",0.997,102,0,2021-10-07,4.000,2,[Argentina]
1,What's Love Got to Do with It?,What's Love Got to Do with It?,0,"[Romance, Comedy]",en,"[English, Urdu, Portuguese]",,Two childhood friends now in their thirties mu...,18.344,109,10898395,2023-01-26,6.100,153,"[France, United Kingdom]"
2,You Can Live Forever,You Can Live Forever,0,"[Drama, Romance]",en,"[English, French]",https://gooddeedentertainment.com/you-can-live...,"When Jaime, a gay teenager, is sent to live in...",5.922,96,15055,2023-03-24,6.800,28,"[Canada, United States of America]"
3,A Tourist's Guide to Love,A Tourist's Guide to Love,0,"[Romance, Comedy]",en,"[English, Vietnamese]",https://www.netflix.com/title/81424906,"After an unexpected break up, a travel executi...",23.381,96,0,2023-04-21,6.365,144,[United States of America]
4,Other People's Children,Les Enfants des autres,0,"[Drama, Comedy]",fr,"[English, French]",https://www.wildbunch.biz/movie/other-peoples-...,"Rachel loves her life, her students, her frien...",13.638,104,84178,2022-09-21,6.900,169,[France]


### Merge and Clean the Data for Export

In [162]:
# Merge the New York Times reviews and TMDB DataFrames on title
merged_movie_df = pd.merge(tmdb_movies_df, reviews_df)
merged_movie_df.head()

,title,original_title,budget,genre,language,spoken_language,homepage,overview,popularity,runtime,...,headline.main,headline.kicker,headline.content_kicker,headline.print_headline,headline.name,headline.seo,headline.sub,byline.original,byline.person,byline.organization
0,The Attachment Diaries,El apego,0,"[Drama, Mystery, Thriller, Horror]",es,[Spanish],,"Argentina, 1970s. A desperate young woman goes...",0.997,102,...,"‘The Attachment Diaries’ Review: Love, Sick",None,None,The Attachment Diaries,None,None,None,By Jeannette Catsoulis,"[{'firstname': 'Jeannette', 'middlename': None...",None
1,You Can Live Forever,You Can Live Forever,0,"[Drama, Romance]",en,"[English, French]",https://gooddeedentertainment.com/you-can-live...,"When Jaime, a gay teenager, is sent to live in...",5.922,96,...,‘You Can Live Forever’ Review: Do You Love Me ...,None,None,You Can Live Forever,None,None,None,By Elisabeth Vincentelli,"[{'firstname': 'Elisabeth', 'middlename': None...",None
2,One True Loves,One True Loves,0,"[Romance, Drama, Comedy]",en,[English],,Emma and Jesse are living the perfect life tog...,17.637,100,...,‘One True Loves’ Review: A Romance Lost at Sea,None,None,One True Loves,None,None,None,By Brandon Yu,"[{'firstname': 'Brandon', 'middlename': None, ...",None
3,The Lost Weekend: A Love Story,The Lost Weekend: A Love Story,0,[Documentary],en,[English],https://thelostweekendmovie.com/,May Pang lovingly recounts her life in rock & ...,1.484,95,...,‘The Lost Weekend: A Love Story’ Review: When ...,None,None,The Lost Weekend: A Love Story,None,None,None,By Glenn Kenny,"[{'firstname': 'Glenn', 'middlename': None, 'l...",None
4,A Thousand and One,A Thousand and One,0,"[Drama, Crime]",en,"[English, Spanish, Portuguese]",https://www.focusfeatures.com/a-thousand-and-one/,Struggling but unapologetically living on her ...,16.465,116,...,‘A Thousand and One’ Review: A New York Love S...,Critic’s Pick,None,An Unbending Will Meets a Shifting City,None,None,None,By Manohla Dargis,"[{'firstname': 'Manohla', 'middlename': None, ...",None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
134,Hounds of Love,Hounds of Love,0,"[Crime, Drama, Horror]",en,[English],http://www.houndsoflovemovie.com/,When Vicki Maloney is randomly abducted from a...,9.631,108,...,Review: ‘Hounds of Love’ Is Tense and Deadly D...,None,None,Hounds of Love,None,None,None,By Ben Kenigsberg,"[{'firstname': 'Ben', 'middlename': None, 'las...",None
135,The Promise,The Promise,90000000,"[History, Romance, Drama]",en,"[Armenian, German, English, French]",http://www.survivalpictures.org/the-promise/,Set during the last days of the Ottoman Empire...,17.373,133,...,Review: ‘The Promise’ Finds a Love Triangle in...,None,None,"A Love Triangle, Near the Front Lines",None,None,None,By Jeannette Catsoulis,"[{'firstname': 'Jeannette', 'middlename': None...",None
136,The Other Half,The Other Half,0,"[Drama, Romance]",en,[English],http://theotherhalfmovie.com/,A grief-stricken man and a bipolar woman fall ...,1.422,103,...,Review: A Combustible Pair Find Love in ‘The O...,None,None,Review: A Combustible Pair Find Love in ‘The O...,None,None,None,By Andy Webster,"[{'firstname': 'Andy', 'middlename': None, 'la...",None
137,The Ottoman Lieutenant,The Ottoman Lieutenant,40000000,"[Romance, Drama, War]",en,"[Turkish, English]",https://www.uphe.com/movies/the-ottoman-lieute...,"Lillie, a determined American woman, ventures ...",10.718,111,...,"Review: Love as the World Wars, in ‘The Ottoma...",None,None,"Review: Love as the World Wars, in ‘The Ottoma...",None,None,None,By Neil Genzlinger,"[{'firstname': 'Neil', 'middlename': None, 'la...",None


In [163]:
# Remove list brackets and quotation marks on the columns containing lists
# Create a list of the columns that need fixing
columns_to_fix = []

for column in merged_movie_df.columns:
    first_entry = str(merged_movie_df[f"{column}"][0])
    if "[" in first_entry \
        or "]" in first_entry \
        or "{" in first_entry \
        or "}" in first_entry \
        or '"' in first_entry \
        or "'" in first_entry:
        columns_to_fix.append(column)
        
# Create a list of characters to remove
characters_to_remove = ["[", "]", "{", "}", '"', "'"]

# Loop through the list of columns to fix
for column in columns_to_fix:
    # Convert the column to type 'str'
    merged_movie_df[f"{column}"] = merged_movie_df[f"{column}"].apply(str)

    # Loop through characters to remove
    for character in characters_to_remove:
        merged_movie_df[f"{column}"] = merged_movie_df[f"{column}"].apply(lambda x: x.replace(character, ""))

# Display the fixed DataFrame
merged_movie_df.head()


,title,original_title,budget,genre,language,spoken_language,homepage,overview,popularity,runtime,...,headline.main,headline.kicker,headline.content_kicker,headline.print_headline,headline.name,headline.seo,headline.sub,byline.original,byline.person,byline.organization
0,The Attachment Diaries,El apego,0,"Drama, Mystery, Thriller, Horror",es,Spanish,,"Argentina, 1970s. A desperate young woman goes...",0.997,102,...,"‘The Attachment Diaries’ Review: Love, Sick",None,None,The Attachment Diaries,None,None,None,By Jeannette Catsoulis,"firstname: Jeannette, middlename: None, lastna...",None
1,You Can Live Forever,You Can Live Forever,0,"Drama, Romance",en,"English, French",https://gooddeedentertainment.com/you-can-live...,"When Jaime, a gay teenager, is sent to live in...",5.922,96,...,‘You Can Live Forever’ Review: Do You Love Me ...,None,None,You Can Live Forever,None,None,None,By Elisabeth Vincentelli,"firstname: Elisabeth, middlename: None, lastna...",None
2,One True Loves,One True Loves,0,"Romance, Drama, Comedy",en,English,,Emma and Jesse are living the perfect life tog...,17.637,100,...,‘One True Loves’ Review: A Romance Lost at Sea,None,None,One True Loves,None,None,None,By Brandon Yu,"firstname: Brandon, middlename: None, lastname...",None
3,The Lost Weekend: A Love Story,The Lost Weekend: A Love Story,0,Documentary,en,English,https://thelostweekendmovie.com/,May Pang lovingly recounts her life in rock & ...,1.484,95,...,‘The Lost Weekend: A Love Story’ Review: When ...,None,None,The Lost Weekend: A Love Story,None,None,None,By Glenn Kenny,"firstname: Glenn, middlename: None, lastname: ...",None
4,A Thousand and One,A Thousand and One,0,"Drama, Crime",en,"English, Spanish, Portuguese",https://www.focusfeatures.com/a-thousand-and-one/,Struggling but unapologetically living on her ...,16.465,116,...,‘A Thousand and One’ Review: A New York Love S...,Critic’s Pick,None,An Unbending Will Meets a Shifting City,None,None,None,By Manohla Dargis,"firstname: Manohla, middlename: None, lastname...",None


In [164]:
# Drop "byline.person" column
merged_movie_df = merged_movie_df.drop(["byline.person"], axis=1)

,title,original_title,budget,genre,language,spoken_language,homepage,overview,popularity,runtime,...,word_count,headline.main,headline.kicker,headline.content_kicker,headline.print_headline,headline.name,headline.seo,headline.sub,byline.original,byline.organization
0,The Attachment Diaries,El apego,0,"Drama, Mystery, Thriller, Horror",es,Spanish,,"Argentina, 1970s. A desperate young woman goes...",0.997,102,...,295,"‘The Attachment Diaries’ Review: Love, Sick",None,None,The Attachment Diaries,None,None,None,By Jeannette Catsoulis,None
1,You Can Live Forever,You Can Live Forever,0,"Drama, Romance",en,"English, French",https://gooddeedentertainment.com/you-can-live...,"When Jaime, a gay teenager, is sent to live in...",5.922,96,...,294,‘You Can Live Forever’ Review: Do You Love Me ...,None,None,You Can Live Forever,None,None,None,By Elisabeth Vincentelli,None
2,One True Loves,One True Loves,0,"Romance, Drama, Comedy",en,English,,Emma and Jesse are living the perfect life tog...,17.637,100,...,320,‘One True Loves’ Review: A Romance Lost at Sea,None,None,One True Loves,None,None,None,By Brandon Yu,None
3,The Lost Weekend: A Love Story,The Lost Weekend: A Love Story,0,Documentary,en,English,https://thelostweekendmovie.com/,May Pang lovingly recounts her life in rock & ...,1.484,95,...,327,‘The Lost Weekend: A Love Story’ Review: When ...,None,None,The Lost Weekend: A Love Story,None,None,None,By Glenn Kenny,None
4,A Thousand and One,A Thousand and One,0,"Drama, Crime",en,"English, Spanish, Portuguese",https://www.focusfeatures.com/a-thousand-and-one/,Struggling but unapologetically living on her ...,16.465,116,...,971,‘A Thousand and One’ Review: A New York Love S...,Critic’s Pick,None,An Unbending Will Meets a Shifting City,None,None,None,By Manohla Dargis,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
134,Hounds of Love,Hounds of Love,0,"Crime, Drama, Horror",en,English,http://www.houndsoflovemovie.com/,When Vicki Maloney is randomly abducted from a...,9.631,108,...,280,Review: ‘Hounds of Love’ Is Tense and Deadly D...,None,None,Hounds of Love,None,None,None,By Ben Kenigsberg,None
135,The Promise,The Promise,90000000,"History, Romance, Drama",en,"Armenian, German, English, French",http://www.survivalpictures.org/the-promise/,Set during the last days of the Ottoman Empire...,17.373,133,...,455,Review: ‘The Promise’ Finds a Love Triangle in...,None,None,"A Love Triangle, Near the Front Lines",None,None,None,By Jeannette Catsoulis,None
136,The Other Half,The Other Half,0,"Drama, Romance",en,English,http://theotherhalfmovie.com/,A grief-stricken man and a bipolar woman fall ...,1.422,103,...,251,Review: A Combustible Pair Find Love in ‘The O...,None,None,Review: A Combustible Pair Find Love in ‘The O...,None,None,None,By Andy Webster,None
137,The Ottoman Lieutenant,The Ottoman Lieutenant,40000000,"Romance, Drama, War",en,"Turkish, English",https://www.uphe.com/movies/the-ottoman-lieute...,"Lillie, a determined American woman, ventures ...",10.718,111,...,267,"Review: Love as the World Wars, in ‘The Ottoma...",None,None,"Review: Love as the World Wars, in ‘The Ottoma...",None,None,None,By Neil Genzlinger,None


In [165]:
# Delete duplicate rows and reset index
merged_movie_df = merged_movie_df.drop_duplicates()
merged_movie_df.head()

,title,original_title,budget,genre,language,spoken_language,homepage,overview,popularity,runtime,...,word_count,headline.main,headline.kicker,headline.content_kicker,headline.print_headline,headline.name,headline.seo,headline.sub,byline.original,byline.organization
0,The Attachment Diaries,El apego,0,"Drama, Mystery, Thriller, Horror",es,Spanish,,"Argentina, 1970s. A desperate young woman goes...",0.997,102,...,295,"‘The Attachment Diaries’ Review: Love, Sick",None,None,The Attachment Diaries,None,None,None,By Jeannette Catsoulis,None
1,You Can Live Forever,You Can Live Forever,0,"Drama, Romance",en,"English, French",https://gooddeedentertainment.com/you-can-live...,"When Jaime, a gay teenager, is sent to live in...",5.922,96,...,294,‘You Can Live Forever’ Review: Do You Love Me ...,None,None,You Can Live Forever,None,None,None,By Elisabeth Vincentelli,None
2,One True Loves,One True Loves,0,"Romance, Drama, Comedy",en,English,,Emma and Jesse are living the perfect life tog...,17.637,100,...,320,‘One True Loves’ Review: A Romance Lost at Sea,None,None,One True Loves,None,None,None,By Brandon Yu,None
3,The Lost Weekend: A Love Story,The Lost Weekend: A Love Story,0,Documentary,en,English,https://thelostweekendmovie.com/,May Pang lovingly recounts her life in rock & ...,1.484,95,...,327,‘The Lost Weekend: A Love Story’ Review: When ...,None,None,The Lost Weekend: A Love Story,None,None,None,By Glenn Kenny,None
4,A Thousand and One,A Thousand and One,0,"Drama, Crime",en,"English, Spanish, Portuguese",https://www.focusfeatures.com/a-thousand-and-one/,Struggling but unapologetically living on her ...,16.465,116,...,971,‘A Thousand and One’ Review: A New York Love S...,Critic’s Pick,None,An Unbending Will Meets a Shifting City,None,None,None,By Manohla Dargis,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
134,Hounds of Love,Hounds of Love,0,"Crime, Drama, Horror",en,English,http://www.houndsoflovemovie.com/,When Vicki Maloney is randomly abducted from a...,9.631,108,...,280,Review: ‘Hounds of Love’ Is Tense and Deadly D...,None,None,Hounds of Love,None,None,None,By Ben Kenigsberg,None
135,The Promise,The Promise,90000000,"History, Romance, Drama",en,"Armenian, German, English, French",http://www.survivalpictures.org/the-promise/,Set during the last days of the Ottoman Empire...,17.373,133,...,455,Review: ‘The Promise’ Finds a Love Triangle in...,None,None,"A Love Triangle, Near the Front Lines",None,None,None,By Jeannette Catsoulis,None
136,The Other Half,The Other Half,0,"Drama, Romance",en,English,http://theotherhalfmovie.com/,A grief-stricken man and a bipolar woman fall ...,1.422,103,...,251,Review: A Combustible Pair Find Love in ‘The O...,None,None,Review: A Combustible Pair Find Love in ‘The O...,None,None,None,By Andy Webster,None
137,The Ottoman Lieutenant,The Ottoman Lieutenant,40000000,"Romance, Drama, War",en,"Turkish, English",https://www.uphe.com/movies/the-ottoman-lieute...,"Lillie, a determined American woman, ventures ...",10.718,111,...,267,"Review: Love as the World Wars, in ‘The Ottoma...",None,None,"Review: Love as the World Wars, in ‘The Ottoma...",None,None,None,By Neil Genzlinger,None


In [166]:
# Export data to CSV without the index
merged_movie_df.to_csv("nyt_tmdb_movie_data.csv", index=False)